In [ ]:
import ants
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pickle
from skimage.io import imread

from zebrafishframework import ants_cmd
from zebrafishframework import io
from zebrafishframework import regtools

regtools = reload(regtools)

In [ ]:
rois_a = pickle.load(open('/Users/koesterlab/registered/control/fish21_6dpf_medium_rois.pickle', 'rb'))
rois_b = pickle.load(open('/Users/koesterlab/registered/control/fish39_6dpf_medium_rois.pickle', 'rb'))

std_dev_a = '/Users/koesterlab/registered/control/fish21_6dpf_medium_std_div.tif'
std_dev_b = '/Users/koesterlab/registered/control/fish39_6dpf_medium_std_div.tif'

In [ ]:
def to_nda(rois):
    # FIXME!!!
    return np.array([[v['y'], v['x'], v['z']] for v in rois])
rois_list = list(map(to_nda, [rois_a, rois_b]))
std_dev_list = [ants.image_read(fn) for fn in [std_dev_a, std_dev_b]]

def filt(p):
    return p[1] < 1000

rois_list = [np.array(list(filter(filt, l))) for l in rois_list]

In [ ]:
for img in std_dev_list:
    plt.figure(figsize=(12,12))
    plt.imshow(img.numpy()[:,:,1].swapaxes(0, 1))

In [ ]:
from skimage.draw import circle

def make_image(nda, radius=20, shape=(21, 1024, 1024)):
    img = np.zeros(shape, np.uint8)
    for p in nda:
        x, y, z = p
        c = circle(y, x, radius, shape[1:])
        img[z][c] = 255
    return img

def to_ants(nda):
    return ants.from_numpy(nda.swapaxes(0, 2))

cell_patterns = list(map(lambda e: to_ants(make_image(e)), rois_list))

for i, cp in enumerate(cell_patterns):
    ants.image_write(cp, '/Users/koesterlab/bla/cell_patterns_%02d.nrrd' % i)
    plt.figure(figsize=(12, 12))
    plt.imshow(cell_patterns[0][:,:,10].swapaxes(0, 1))


In [ ]:
"""
res = ants.registration(cell_patterns[0], 
                        cell_patterns[1], 
                        type_of_transform='Affine',
                        verbose=True)
warped = res['warpedmovout'].swapaxes(0, 2)
"""
warped, transforms, zshift = regtools.planewise_affine(cell_patterns[0], cell_patterns[1], True)

plt.figure(figsize=(12, 12))
plt.imshow(cell_patterns[0][:,:,10].swapaxes(0, 1))
plt.figure(figsize=(12, 12))
plt.imshow(warped[10])

In [ ]:
"""
t = ants.read_transform(res['fwdtransforms'][0])
def apply(p):
    img = cell_patterns[1]
    phys = ants.transform_index_to_physical_point(img, p)
    trans = np.array(list(ants.apply_ants_transform_to_point(t, phys)), dtype=np.int32)
    back = ants.transform_physical_point_to_index(img, trans)
    return np.array(back, dtype=np.int32)
transformed_rois = np.array(list(map(apply, rois_list[1])))
"""


In [ ]:
transformed_rois = regtools.transform_planewise_points(rois_list[1], transforms, zshift)

In [ ]:
def filter_shape(p, shape):
    if np.any(p < 0):
        return False
    if np.any(p > shape):
        return False
    return True
transformed_rois = np.array(list(filter(lambda p: filter_shape(p, (1024, 1024, 21)), transformed_rois)))

In [ ]:
args = ants_zff.AntsArguments(input_file=std_dev_b, reference=std_dev_a, params=ants_zff.get_default_params()[:2])
args.output_folder = os.path.join('/Users/koesterlab/ants_tmp', args.output_folder)
res = ants_zff.run_antsreg(args)

In [ ]:
def render_rois(ndas, colors, matching=None, plane=10, shape=(1024, 1024)):
    image = np.zeros(shape + (3,), dtype=np.uint8)
    for nda, color in zip(ndas, colors):
        color = np.array(color, dtype=np.uint8)
        for roi in nda:
            x, y, z = roi
            if z == plane:
                image[y, x] += color
    
    if matching:
        pass
    
    return image

image = render_rois([rois_list[0], transformed_rois], [(0, 255, 0), (255, 0, 255)], plane=16)
plt.figure(figsize=(12, 12))
plt.imshow(image)

In [ ]:
def initial_solution(ndas):
    dim = max(map(np.alen, ndas))
    num = np.alen(ndas)
    sol = np.full((dim, num), -1, dtype=np.int32)
    for i, rois in enumerate(ndas):
        
    return sol

In [ ]:
initial_solution([a, b])